<a href="https://colab.research.google.com/github/Prags21/SentimentAnalysis/blob/master/World_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# pylint: skip-file
import csv
import pandas as pd
import collections

table=pd.read_csv('country.csv', encoding = "ISO-8859-1")
a=[]
for ind,item in table.iterrows():
    a.append(item['Country'])
counts=collections.Counter(a)
print(counts)
keys=['Country','Tweets Count']
with open('result.csv', 'a') as csvFile:
    dict_writer = csv.DictWriter(csvFile, fieldnames=keys)
    dict_writer.writeheader()
    for ind in counts:
        dict_={}
        dict_ = {'Country': ind,
             'Tweets Count': counts[ind]
                }
        dict_writer.writerow(dict_)

In [0]:
# Import libraries
import pandas as pd
import folium
 
# Load the shape of the zone (US states)
# Find the original file here: https://github.com/python-visualization/folium/tree/master/examples/data
# You have to download this file and set the directory where you saved it
 
# Load the unemployment value of each state
# Find the original file here: https://github.com/python-visualization/folium/tree/master/examples/data
state_data = pd.read_csv('result.csv')
 
# Initialize the map:
world_map = folium.Map(location=[0, 0], zoom_start=2)
bins = list(state_data['Tweets Count'].quantile([0, 0.1, 0.2, 0.3,0.5,0.6,0.7,0.8,0.9, 1]))

# Add the color for the chloropleth:
folium.Choropleth(
 geo_data='world-map.json',
 name='choropleth',
 data=state_data,
 columns=['Country', 'Tweets Count'],
 key_on='feature.properties.name',
 fill_color='YlGn',
 nan_fill_color='white',  
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Awareness Rate (%)'
).add_to(world_map)
folium.LayerControl().add_to(world_map)
 
# Save to html
world_map.save('countries_distribution.html')
world_map

In [0]:
from branca.colormap import LinearColormap
import json

map_data = pd.read_csv('result.csv')


map_dict = map_data.set_index('Country')['Tweets Count'].to_dict()

color_scale = LinearColormap(['yellow','red'], vmin = min(map_dict.values()), vmax = max(map_dict.values()))

def get_color(feature):
    value = map_dict.get(feature['properties']['name'])
    if value is None:
        return '#8c8c8c' # MISSING -> gray
    else:
        return color_scale(value)

m = folium.Map(
    location = [50, 15], 
    zoom_start = 4
)

folium.GeoJson(
    data ='world-map.json',
    style_function = lambda feature: {
        'fillColor': get_color(feature),
        'fillOpacity': 0.7,
        'color' : 'black',
        'weight' : 1,
        'legend_name':'Unemployment Rate (%)'

    }    
).add_to(m)
m